# Practical Exercise: Data Preparation and Visualisation

## Exercise I. Data Loading and Preparation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os, glob
import re
from datetime import datetime, date, time
from six import iteritems
%load_ext autoreload
%autoreload 2

###  1. Data Reading and Exploration

### Initialize Class and Read files
In blond.py file, you can find the Blond class which handles data input, output and calibration. When the Blond class is initiated, it tries to acquire the data, whose interval is specified as an input. 

**Note that:** 
* Your data should be under **./data/** folder. For example **./data/clear** and **./data/medal-2**. 

* It only reads h5py file names and extracts the structure of the data, i.e. number of medals and signal names under each device, which is not actually reading the data. 

* From this point on, using this Blond object one can not read any other time which is not in the interval of the data itself. For example, your latest clear hdf5 file is 02:15:20, when you ask for clear data between 02:20:25-02:20:30 you wont get any data. But you are still able to get data for 02:18:10-02:20:20, since clear data contains 5 minutes of data in a file.

In [ ]:
from blond import Blond,get_time_diff
blond = Blond(date(2016,10,5))

Checking the retrieved files

In [ ]:
blond.list_files()

Checking the data structure

See the possible time ranges for each device. Note that, all the times are inclusive. If you see a time of *medal-1* for *latest* *2:17:12* , you will get the data until the end of 12.th second.

In [ ]:
blond.time_limits

### Example Data Read

In order to read a data interval, one needs to specify **device**("clear", "medal-1"...), **signal**("current1", "current2"...), start and end times. The time parameters here should be present in the specified device data. You can read either with or without calibration and centering. 

**Note:** While start time(start_ts) is inclusive, end time(end_ts) is exclusive. So for the example below, you will get the data of whole 0:50:0, so 0:50:1 is not included.

In [ ]:
the_data_uncalibrated = blond.read_data(
    device="medal-1",
    signal="current1",
    start_ts=time(0,50,0), 
    end_ts=time(0,50,2),
    calibrate = False,
    center = False,
)

In [ ]:
fig = plt.figure(figsize=[40, 40])
x = np.linspace(0,2,the_data_uncalibrated.shape[0])
plt.plot(x,the_data_uncalibrated)
fig.suptitle("Medal-1 Data Calibrated For Current1 Signal Between 0:50:00-0:50:02",fontsize=40)
plt.savefig("exercise1-uncalibrated.svg",format="svg")

### Exploration


Signals acquisited by MEDAL

In [ ]:
blond.get_data_structure()["medal-1"]

Signals acquisited by CLEAR

In [ ]:
blond.get_data_structure()["clear"]

### Centering and calibrating

When *read_data* function of the Blond class is called, the data is calibrated automatically. So we do not need an extra function call here.

**Note:** If you do not want the data centered and calibrated, you can call *read_data* with the **calibrate** = *False* and **center**=*False*parameter. Otherwise it will be always calibrated and centered.

You can create calibrated and not calibrated data here.

In [ ]:
the_data_calibrated_notcentered = blond.read_data(
    device="medal-1",
    signal="current1",
    start_ts=time(0,50,0), 
    end_ts=time(0,50,2),
    center=False,
    calibrate=True,
)
the_data_calibrated_centered = blond.read_data(
    device="medal-1",
    signal="current1",
    start_ts=time(0,50,0), 
    end_ts=time(0,50,2),
    calibrate = True,
    center = True,
)

In [ ]:
fig = plt.figure(figsize=[40, 40])
x = np.linspace(0,2,the_data_calibrated_notcentered.shape[0])
plt.plot(x,the_data_calibrated_notcentered)
fig.suptitle("Medal-1 Data not calibrated for current1 Signal Between 0:50:00-0:50:02",fontsize=40)
plt.savefig("exercise2-noncentered.svg",format="svg")

In [ ]:
fig = plt.figure(figsize=[40, 40])
x = np.linspace(0,2,the_data_calibrated_centered.shape[0])
plt.plot(x,the_data_calibrated_centered)
fig.suptitle("Medal-1 Data Calibrated For Current1 Signal Between 0:50:00-0:50:02",fontsize=40)
plt.savefig("exercise2-centered.svg",format="svg")

## Exercise III. Feature Extraction

In order to run the Dash-Plotly application, you need to run **python app.py**. After the server runs, you can connect to the server and use the application.

## Exercise IV. Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

requested_time_start = time(0,0,9)
requested_time_end = time(0,0,10)

temp_data1 = blond.read_data(
    device="medal-3",
    signal="current5",
    start_ts=requested_time_start, 
    end_ts=requested_time_end,
)

temp_data2 = blond.read_data(
    device="medal-3",
    signal="voltage",
    start_ts=requested_time_start, 
    end_ts=requested_time_end,
)

temp_data3 = blond.read_data(
    device="medal-3",
    signal="current5",
    start_ts=time(0,0,9), 
    end_ts=time(0,0,14)
)

temp_data4 = blond.read_data(
    device="medal-3",
    signal="voltage",
    start_ts=time(0,0,9), 
    end_ts=time(0,0,14)
)

#calculate period Root-Mean-Square of the Current
curr_rms = []
for curr in temp_data1:
    curr_rms.append(np.sqrt(np.mean(curr**2)))
    
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(120, 40))


plot1 = ax[0]
t = np.arange(0.0, 1.0, 1/6400)
plot1.plot(t, temp_data1, color='b', label='Current')
plot1.plot(t, curr_rms, color='r', label='Period RMS of Current')
plot1.set_title('Current of Monitor')
plot1.set_xlabel('Time [s]')
plot1.set_ylabel('Current [A]')
plot1.legend()

plot2 = ax[1]
t = np.arange(0.0, 40, 40/6400)
sinewave = np.sin(2*np.pi*t/20)
plot2.plot(t, temp_data2/6400, color='r', label='voltage')
plot2.plot(t, curr_rms, color='b', label='current')
plot2.plot(t, sinewave, color='y', label='pure sine')
plot2.set_title('Waveform Comparison of Monitor')
plot2.set_xlabel('Time [ms]')
plot2.legend()

plot3 = ax[2]
t = np.arange(0.1, 5.1, 1/6400)
plot3.plot(t, temp_data3*temp_data4/t, color='b')
plot3.set_title('Power over Time of Monitor')
plot3.set_xlabel('Time [s]')
plot3.set_ylabel('Power [W]')

plt.subplots_adjust(left=0, wspace=0.4, top=0.8)

plt.savefig("exercise4.svg",format="svg")